In [4]:
import tushare as ts
from typing import Any

import numpy as np
import pandas as pd

pro = ts.pro_api('2026c96ef5fa7fc3241c96baafd638c585284c7fefaa00b93ef0a62c')

def code_to_secid(df : pd.DataFrame , code_col = 'ts_code' , retain = False):
    '''switch old symbol into secid'''
    if code_col not in df.columns.values: return df
    replace_dict = {'T00018' : '600018'}
    df['secid'] = df[code_col].astype(str).str.slice(0, 6).replace(replace_dict)
    df['secid'] = df['secid'].where(df['secid'].str.isdigit() , '-1').astype(int)
    if not retain: del df[code_col]
    return df

# adj0 = pro.query('adj_factor',  trade_date='20240705')


In [17]:
date = 19970103
date_str = str(date)
adj = pro.query('adj_factor',  trade_date=date_str).rename(columns={'adj_factor':'adjfactor'})

quote = pro.daily(trade_date=date_str).rename(columns={'pct_chg':'pctchange','pre_close':'preclose','vol':'volume'})
quote['volume'] = quote['volume'] / 10. # to 10^3
quote['vwap'] = np.where(quote['volume'] == 0 , quote['close'] , quote['amount'] / quote['volume'])

shr = pro.daily_basic(trade_date=date_str).loc[:,['ts_code','trade_date' , 'total_share','float_share','free_share']]
shr.loc[:,['total_share','float_share','free_share']] *= 1e4
shr.loc[shr['free_share'].isna() , 'free_share'] = shr.loc[shr['free_share'].isna() , 'float_share']

limit = pro.stk_limit(trade_date=date_str)
if len(limit) == 0:
    limit = quote.loc[:,['ts_code' , 'trade_date' , 'close']].copy()
    limit['up_limit'] = (limit['close'] * 1.1).round(2)
    limit['down_limit'] = (limit['close'] * 0.9).round(2)
    limit = limit.drop(columns=['close'])

susp = pro.suspend_d(suspend_type='S', trade_date=date_str)

mutual_col = ['ts_code' , 'trade_date']

trade = quote.merge(adj,on=mutual_col,how='left').\
    merge(limit,on=mutual_col,how='left').\
    merge(shr,on=mutual_col,how='left')
trade['status'] = 1.0 * ~trade['ts_code'].isin(susp['ts_code']).fillna(0)
trade['limit'] = 1.0 * (trade['close'] >= trade['up_limit']).fillna(0) - 1.0 * (trade['close'] <= trade['down_limit']).fillna(0)
trade['turn_tt'] = (trade['volume'] / trade['total_share'] * 1e5).fillna(0)
trade['turn_fl'] = (trade['volume'] / trade['float_share'] * 1e5).fillna(0)
trade['turn_fr'] = (trade['volume'] / trade['free_share'] * 1e5).fillna(0)

trade = code_to_secid(trade).set_index('secid').sort_index().reset_index().loc[
    :,['secid', 'adjfactor', 'open', 'high', 'low', 'close', 'amount','volume', 'vwap', 
    'status', 'limit', 'pctchange', 'preclose', 'turn_tt','turn_fl', 'turn_fr']]
trade

,secid,adjfactor,open,high,low,close,amount,volume,vwap,status,limit,pctchange,preclose,turn_tt,turn_fl,turn_fr
0,1,13.968,16.50,16.59,16.18,16.30,118729.5778,7246.2,16.385082,1.0,0.0,-1.21,16.50,0.700411,1.014273,27.344151
1,2,5.121,10.60,11.20,10.38,10.45,66558.8782,6157.9,10.808697,1.0,0.0,0.00,10.45,1.942263,3.557413,21.992500
2,3,3.833,8.74,9.30,8.70,8.80,89765.3515,9964.3,9.008696,1.0,0.0,1.27,8.69,2.988391,5.196384,41.270681
3,4,3.112,6.15,6.20,5.87,6.00,9643.5748,1585.7,6.081588,1.0,0.0,-3.54,6.22,1.888262,3.806542,31.714000
4,5,2.275,9.96,10.30,9.60,9.70,44827.8666,4502.3,9.956659,1.0,0.0,-1.02,9.80,1.694824,3.574279,3.574279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,600896,1.000,7.66,7.88,7.35,7.60,9195.5540,1203.1,7.643217,1.0,0.0,-0.78,7.66,0.721716,2.885132,2.885132
512,600897,1.000,17.40,17.40,16.80,16.84,12697.5670,746.7,17.004911,1.0,0.0,-3.22,17.40,0.691389,2.765556,2.765556
513,600898,1.320,8.00,8.05,7.47,7.60,11454.2910,1471.7,7.783034,1.0,0.0,-5.00,8.00,1.094004,2.198635,2.858225
514,600899,1.000,11.20,11.75,11.10,11.23,8995.8720,793.3,11.339811,1.0,0.0,0.45,11.18,0.717269,2.644333,2.644333


In [1]:
import tushare as ts
from src.data.tushare.basic import pro , code_to_secid , DateFetcher

import numpy as np
import pandas as pd

class MinQuote(DateFetcher):
    '''minute Quote'''
    def db_src(self): return 'trade_ts'
    def db_key(self): return 'min'    
    def get_data(self , date : int):
        date_str = str(date)
        adj = pro.query('adj_factor',  trade_date=date_str).rename(columns={'adj_factor':'adjfactor'})

        quote = pro.daily(trade_date=date_str).rename(columns={'pct_chg':'pctchange','pre_close':'preclose','vol':'volume'})
        quote['volume'] = quote['volume'] / 10. # to 10^3
        quote['vwap'] = np.where(quote['volume'] == 0 , quote['close'] , quote['amount'] / quote['volume'])

        shr = pro.daily_basic(trade_date=date_str).loc[:,['ts_code','trade_date' , 'total_share','float_share','free_share']]
        shr.loc[:,['total_share','float_share','free_share']] *= 1e4
        shr.loc[shr['free_share'].isna() , 'free_share'] = shr.loc[shr['free_share'].isna() , 'float_share']

        limit = pro.stk_limit(trade_date=date_str)
        if len(limit) == 0:
            limit = quote.loc[:,['ts_code' , 'trade_date' , 'close']].copy()
            limit['up_limit'] = (limit['close'] * 1.1).round(2)
            limit['down_limit'] = (limit['close'] * 0.9).round(2)
            limit = limit.drop(columns=['close'])

        susp = pro.suspend_d(suspend_type='S', trade_date=date_str)

        mutual_col = ['ts_code' , 'trade_date']

        trade = quote.merge(adj,on=mutual_col,how='left').\
            merge(limit,on=mutual_col,how='left').\
            merge(shr,on=mutual_col,how='left')
        trade['status'] = 1.0 * ~trade['ts_code'].isin(susp['ts_code']).fillna(0)
        trade['limit'] = 1.0 * (trade['close'] >= trade['up_limit']).fillna(0) - 1.0 * (trade['close'] <= trade['down_limit']).fillna(0)
        trade['turn_tt'] = (trade['volume'] / trade['total_share'] * 1e5).fillna(0)
        trade['turn_fl'] = (trade['volume'] / trade['float_share'] * 1e5).fillna(0)
        trade['turn_fr'] = (trade['volume'] / trade['free_share'] * 1e5).fillna(0)

        trade = code_to_secid(trade).set_index('secid').sort_index().reset_index().loc[
            :,['secid', 'adjfactor', 'open', 'high', 'low', 'close', 'amount','volume', 'vwap', 
            'status', 'limit', 'pctchange', 'preclose', 'turn_tt','turn_fl', 'turn_fr']]
        return trade
    
'''
df = ts.pro_bar(ts_code='600000.SH', api = pro , freq='1min', 
                start_date='2024-11-05 00:00:00', 
                end_date='2024-11-05 17:00:00')

df
'''


"\ndf = ts.pro_bar(ts_code='600000.SH', api = pro , freq='1min', \n                start_date='2024-11-05 00:00:00', \n                end_date='2024-11-05 17:00:00')\n\ndf\n"

In [1]:
import tushare as ts

from src.data.tushare.basic import pro

In [1]:
from src.data.tushare.download import TushareDownloader

TushareDownloader.proceed()

Calendar Already Updated at 20241109
ChangeName Already Updated at 20241109
Description Already Updated at 20241109
SWIndustry Already Updated at 20241109
THSConcept Already Updated at 20241031
DailyLimit Updating trade_ts/day_limit at 20241109
DailyMoneyFlow Updating trade_ts/day_moneyflow at 20241109
DailyQuote Updating trade_ts/day at 20241109
DailyValuation Updating trade_ts/day_val at 20241109
BalanceSheet Updating financial_ts/balance at 19970331
BalanceSheet Updating financial_ts/balance at 19970630
BalanceSheet Updating financial_ts/balance at 19970930
BalanceSheet Updating financial_ts/balance at 19971231
BalanceSheet Updating financial_ts/balance at 19980331
BalanceSheet Updating financial_ts/balance at 19980630
BalanceSheet Updating financial_ts/balance at 19980930
BalanceSheet Updating financial_ts/balance at 19981231
BalanceSheet Updating financial_ts/balance at 19990331
BalanceSheet Updating financial_ts/balance at 19990630
BalanceSheet Updating financial_ts/balance at 19

In [1]:
from src.data.tushare.download.analyst import AnalystReport
ana = AnalystReport()
ana.update()

AnalystReport Updating analyst_ts/report from 20100103 to 20100221
20100103 20100221
AnalystReport Updating analyst_ts/report from 20100222 to 20100412
20100222 20100412
AnalystReport Updating analyst_ts/report from 20100413 to 20100601
20100413 20100601
AnalystReport Updating analyst_ts/report from 20100602 to 20100721
20100602 20100721
AnalystReport Updating analyst_ts/report from 20100722 to 20100909
20100722 20100909
AnalystReport Updating analyst_ts/report from 20100910 to 20101029
20100910 20101029
AnalystReport Updating analyst_ts/report from 20101030 to 20101218
20101030 20101218
AnalystReport Updating analyst_ts/report from 20101219 to 20110206
20101219 20110206
AnalystReport Updating analyst_ts/report from 20110207 to 20110328
20110207 20110328
AnalystReport Updating analyst_ts/report from 20110329 to 20110517
20110329 20110517
AnalystReport Updating analyst_ts/report from 20110518 to 20110706
20110518 20110706
AnalystReport Updating analyst_ts/report from 20110707 to 2011082

In [2]:
dfs = []
offset = 0
limit = 2000
while True:
    df = pro.report_rc(start_date='20220528' , end_date = '20220531' , limit = limit , offset = offset)
    if len(df) == 0: break
    dfs.append(df)
    offset += limit



In [3]:
import pandas as pd
pd.concat(dfs)

,ts_code,name,report_date,report_title,report_type,classify,org_name,author_name,quarter,op_rt,...,tp,np,eps,pe,rd,roe,ev_ebitda,rating,max_price,min_price
0,300394.SZ,天孚通信,20220531,天孚通信：多元创新打开新空间，公司持续高成长可期,点评,首次评级,银河证券,赵良毕,2024Q4,226763.0,...,76419.0,67537.00,1.720,15.50,NaN,17.13,12.58,推荐,NaN,NaN
1,300394.SZ,天孚通信,20220531,天孚通信：多元创新打开新空间，公司持续高成长可期,点评,首次评级,银河证券,赵良毕,2023Q4,177159.0,...,59791.0,52842.00,1.350,19.81,NaN,16.17,17.02,推荐,NaN,NaN
2,300394.SZ,天孚通信,20220531,天孚通信：多元创新打开新空间，公司持续高成长可期,点评,首次评级,银河证券,赵良毕,2022Q4,136276.0,...,45992.0,40646.00,1.040,25.75,NaN,14.84,23.08,推荐,NaN,NaN
3,300638.SZ,广和通,20220531,广和通：研发能力不断提升，产品突破逐见成效,点评,首次评级,银河证券,赵良毕,2024Q4,1039837.0,...,101696.0,95798.00,1.542,9.79,NaN,22.91,10.10,推荐,NaN,NaN
4,300638.SZ,广和通,20220531,广和通：研发能力不断提升，产品突破逐见成效,点评,首次评级,银河证券,赵良毕,2023Q4,787755.0,...,77042.0,72574.00,1.168,12.92,NaN,22.51,14.21,推荐,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,000983.SZ,山西焦煤,20220528,煤炭开采：预期持续改善，催化板块上行,非个股,一般报告,中泰证券,"杜冲,王璇",2023Q4,NaN,...,NaN,962691.60,2.350,NaN,NaN,NaN,NaN,买入,NaN,NaN
1566,000983.SZ,山西焦煤,20220528,煤炭开采：预期持续改善，催化板块上行,非个股,一般报告,中泰证券,"杜冲,王璇",2022Q4,NaN,...,NaN,954498.48,2.330,NaN,NaN,NaN,NaN,买入,NaN,NaN
1567,300856.SZ,科思股份,20220528,科思股份：全球防晒剂龙头，成就原料国货崛起之路,深度,首次关注,东吴证券,"吴劲草,张家璇",2024Q4,194291.0,...,38600.0,33000.00,2.930,17.67,NaN,13.69,NaN,买入,NaN,NaN
1568,300856.SZ,科思股份,20220528,科思股份：全球防晒剂龙头，成就原料国货崛起之路,深度,首次关注,东吴证券,"吴劲草,张家璇",2023Q4,172951.0,...,32300.0,27600.00,2.450,21.13,NaN,13.28,NaN,买入,NaN,NaN


In [8]:
df = pro.forecast_vip(period='20181231')
df

,ts_code,ann_date,end_date,type,p_change_min,p_change_max,net_profit_min,net_profit_max,last_parent_net,first_ann_date,summary,change_reason,update_flag
0,600393.SH,20190429,20181231,预减,-77.4400,-72.9200,26350.0,31620.0,116785.7900,20190130,预计净利润26350-31620万,（一）前次业绩预告数据测算过程中，由于财务部门会计政策理解偏差，非经常性损益扣除了商誉、无形...,0
1,603996.SH,20190429,20181231,首亏,-158.2012,-127.3888,-8500.0,-4000.0,14604.5000,20190429,预计净利润-8500~-4000万,1.2018年下半年以来，控股股东股票质押率较高，导致公司面临一些银行信贷方面的资金压力，再...,0
2,600687.SH,20190427,20181231,首亏,-314.6186,-310.9499,-117000.0,-115000.0,54515.3100,20190131,预计净利润-117000~-115000万,公司本次业绩预亏主要原因如下：（一）报告期内，受到国际、国内经济环境及负面报道等多方面因素影...,0
3,002359.SZ,20190427,20181231,首亏,-813.1968,-575.4646,-155214.0,-103476.0,21763.1300,20181023,预计净利润-155214~-103476万,本次业绩预告修正的主要原因系：1、公司债务逾期导致诉讼事项及公司银行账户被冻结，经公司与客户...,0
4,600634.SH,20190426,20181231,首亏,-17107.0000,-17107.0000,-550900.0,-550900.0,3239.4500,20181031,预计净利润-550900万,（一）关于公司或有负债的计提.出于谨慎性原则，公司对上述或有借款计提了预计负债，金额约为26...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,300131.SZ,20180911,20181231,略增,32.8600,60.8400,19000.0,23000.0,14300.3200,20180911,预计净利润19000-23000万,1、受益于云服务、汽车电子、集成电路市场规模的持续扩大，5G通信迎来建设期以及智能硬件技术走...,1
3135,601068.SH,20180830,20181231,续盈,NaN,NaN,NaN,NaN,59120.8852,20180830,续盈,截至本上市公告书签署日，公司各项业务状况正常，未出现影响公司经营的重大不利因素，综合公司所处...,1
3136,002933.SZ,20180807,20181231,略增,2.9500,20.1100,12000.0,14000.0,11655.7900,20180807,预计净利润12000-14000万,公司围绕主营业务，稳步推进年度经营指标，主营业务收入同比增加，利润相应增加。,0
3137,603590.SH,20180806,20181231,略减,-47.1600,-45.9100,25300.0,25900.0,47881.7100,20180806,预计净利润25300-25900万,None,0


In [5]:
df

,ts_code,ann_date,f_ann_date,end_date,report_type,comp_type,end_type,basic_eps,diluted_eps,total_revenue,...,withdra_legal_pubfund,withdra_biz_devfund,withdra_rese_fund,withdra_oth_ersu,workers_welfare,distr_profit_shrhder,prfshare_payable_dvd,comshare_payable_dvd,capit_comstock_div,update_flag
0,605028.SH,20210601,20210601,20181231,1,1,4,NaN,NaN,2.526022e+08,...,None,None,None,None,None,None,None,None,None,1
1,605162.SH,20210616,20210616,20181231,1,1,4,0.35,0.35,6.194405e+08,...,None,None,None,None,None,None,None,None,None,1
2,001216.SZ,20210831,20210831,20181231,1,1,4,0.57,0.57,8.447349e+08,...,None,None,None,None,None,None,None,None,None,1
3,605011.SH,20210608,20210608,20181231,1,1,4,0.43,0.43,1.945367e+09,...,None,None,None,None,None,None,None,None,None,1
4,688789.SH,20210621,20210621,20181231,1,1,4,1.79,1.79,4.727597e+08,...,None,None,None,None,None,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,873731.BJ,20190420,20190420,20181231,1,1,4,0.21,0.21,8.363539e+07,...,None,None,None,None,None,None,None,None,None,0
8621,301413.SZ,20220228,20220228,20181231,1,1,4,0.21,0.21,2.589512e+08,...,None,None,None,None,None,None,None,None,None,1
8622,300091.SZ,20190402,20240208,20181231,1,1,4,-0.06,-0.06,1.388244e+09,...,None,None,None,None,None,None,None,None,None,1
8623,301588.SZ,20220331,20220331,20181231,1,1,4,0.05,0.05,3.214190e+08,...,None,None,None,None,None,None,None,None,None,1


In [12]:
pro.daily(start_date='20240708')

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,003032.SZ,20240708,9.47,9.50,8.74,8.88,9.47,-0.59,-6.2302,195958.79,177127.325
1,002688.SZ,20240708,3.68,3.70,3.59,3.60,3.71,-0.11,-2.9650,66932.00,24218.867
2,688238.SH,20240708,4.56,4.56,4.30,4.32,4.56,-0.24,-5.2632,54944.70,24271.305
3,605090.SH,20240708,28.76,28.86,28.07,28.45,28.87,-0.42,-1.4548,43069.40,122560.860
4,600611.SH,20240708,2.77,3.04,2.77,2.95,2.78,0.17,6.1151,466175.26,135351.973
...,...,...,...,...,...,...,...,...,...,...,...
5328,002873.SZ,20240708,8.58,8.63,8.21,8.23,8.53,-0.30,-3.5170,26403.99,21990.333
5329,001373.SZ,20240708,23.85,24.15,23.12,23.26,23.91,-0.65,-2.7185,6656.00,15626.912
5330,603018.SH,20240708,8.52,8.55,8.25,8.35,8.56,-0.21,-2.4533,171655.07,143505.269
5331,000797.SZ,20240708,2.35,2.36,2.27,2.28,2.37,-0.09,-3.7975,176813.00,40648.298
